# FINAL PROJECT
# PROJECT-4. РЕШЕНИЕ КОМПЛЕКСНОЙ БИЗНЕС-ЗАДАЧИ

### ЛЕГЕНДА:
#### Продакт-менеджер онлайн-школы MasterMind замечает, что новые курсы не пользуются особой популярностью среди пользователей. Было принято решение помочь повысить интерес пользователей к выбору других курсов.

### ЦЕЛЬ: 
#### Подготовить основу рекомендательной системы.

### ЗАДАЧА:
#### Создать рекомендательную систему в виде таблицы, в которой курсам будет соответствовать по две рекомендации.

### ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ:
#### 1) Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL.
#### 2) Обработать данные средствами Python.
#### 3) Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить отчёт продакт-менеджеру.
#### 4) Проанализировать результаты A/B-теста, проведённого после внедрения фичи, и сделать вывод.

________________________________________________________________________________________________________________________________________________________________________________________________

### ШАГ 1. Ознакомление с датасетом, подготовка и анализ данных с помощью SQL.

#### СОДЕРЖИМОЕ ДАТАСЕТА:
- carts с данными о пользовательских корзинах (дате создания, статусе, id пользователя-владельца и т. д.);
- cart items с данными о курсах, которые пользователи добавили в корзину.

Перед выгрузкой данных полезно ознакомиться с данными при помощи пары вопросов:
1) Какой промежуток времени охватывает данный датасет?

    SELECT
        MIN(purchased_at),
        MAX(purchased_at)
    FROM skillfactory.final.carts c

Ответ: 01.01.2017 - 30.12.2018

2) Сколько клиентов покупали курсы?

    SELECT 
        COUNT (DISTINCT user_id) AS unique_cust
    FROM skillfactory.final.carts c
    JOIN skillfactory.final.cart_items i ON c.id = i.cart_id
    WHERE purchased_at IS NOT NULL
        AND resource_type = 'Course'
        AND state = 'successful'

Ответ: 49006

3) Сколько всего есть различных курсов?

    SELECT 
        COUNT (DISTINCT i.resource_id) AS unique_courses
    FROM skillfactory.final.cart_items i 
    JOIN skillfactory.final.carts c on c.id = i.cart_id
    WHERE resource_type = 'Course'

Ответ: 127

4) Каково среднее число купленных курсов на одного клиента?

    SELECT 
        COUNT (i.resource_id) AS total_sales_courses,
        COUNT (i.resource_id) / COUNT (DISTINCT c.user_id)::DECIMAL AS average_by_users
    FROM skillfactory.final.carts c 
    JOIN skillfactory.final.cart_items i on c.id =  i.cart_id
    WHERE resource_type = 'Course'
        AND c.state = 'successful'

Ответ: 1.44

5) Сколько клиентов купили больше одного курса?

    WITH user_course AS
    (SELECT 
        user_id,
        COUNT (DISTINCT resource_id) AS course_cnt
    FROM final.carts AS c  
    JOIN final.cart_items AS i ON c.id = i.cart_id
    WHERE resource_type = 'Course' 
        AND state = 'successful'
    GROUP BY 1
    ORDER BY 2 desc)

    SELECT
        COUNT (user_id)
    FROM user_course
    WHERE course_cnt > 1

Ответ: 12656

Финальным этапом в работе с SQL станет подготовка файла с данными по продажам курсов в разрезе пользователей, купивших более одного курса.\
За основу берем таблицу с id-пользователя и количеством купленных им курсов (от 1). Запрос выглядит следующим образом:

    WITH count_users AS
    (
        SELECT  
            user_id,
            COUNT (resource_id) AS course_cnt
        FROM skillfactory.final.carts AS c
        JOIN skillfactory.final.cart_items AS i ON i.cart_id = c.id
        WHERE state = 'successful'
            AND resource_type = 'Course'
        GROUP BY  user_id
        HAVING COUNT (DISTINCT resource_id) > 1
    )    
    SELECT  
        DISTINCT count_users.user_id,
        resource_id
    FROM skillfactory.final.carts AS c
    JOIN skillfactory.final.cart_items AS i ON i.cart_id = c.id
    RIGHT JOIN count_users ON c.user_id = count_users.user_id
    WHERE state = 'successful'
        AND resource_type = 'Course'
    ORDER BY count_users.user_id, resource_id


### ШАГ 2. ОБРАБОТКА ДАННЫХ ПРИ ПОМОЩИ PYTHON

Проанализируем полученный список при помощи Python.

1) Импорт необходимых библиотек и считывание файла с данными 

In [38]:
import pandas as pd
import numpy as np
from itertools import combinations
from collections import defaultdict

df = pd.read_csv('data/sql_final.csv')
df.head()

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


2. Обработка данных\
На данном этапе разобьем все покупки курсов по парам, после чего отсортируем их по убыванию.

In [39]:
# Создаем словарь: {user_id: [resource_id1, resource_id2, ...]}
user_courses = df.groupby("user_id")["resource_id"].apply(set).to_dict()

# Создаем словарь для подсчета пар курсов
pair_counts = defaultdict(int)

# Перебираем всех пользователей и их курсы
for courses in user_courses.values():
    # Генерируем все возможные уникальные пары курсов для пользователя
    for pair in combinations(sorted(courses), 2):
        pair_counts[pair] += 1

# Создаем DataFrame из результатов
pairs_df = pd.DataFrame(
    [(pair[0], pair[1], count) for pair, count in pair_counts.items()],
    columns=["first_course", "second_course", "pair_count"],
)

# Сортируем по количеству общих пользователей (по убыванию)
pairs_df = pairs_df.sort_values("pair_count", ascending=False)
pairs_df

,first_course,second_course,pair_count
186,551,566,797
381,515,551,417
205,489,551,311
105,523,551,304
519,566,794,290
...,...,...,...
3973,800,1144,1
3972,489,1185,1
3970,765,814,1
3969,745,814,1


По результатам выведенной таблицы видим, что в покупках клиентов встречается 3989 различных пар курсов.\
А также самая популярная пара (551, 566) встречалась 797 раз. Начинаем строить рекомендательную систему из имеющихся пар курсов.

### ШАГ 3. СОСТАВЛЕНИЕ ИТОГОВОЙ ТАБЛИЦЫ С РЕКОМЕНДАЦИЯМИ

Основная задача - это создать рекомендательную систему в формате таблицы, в которой курсам будет соответствовать по две рекомендации.\
Однако, не все пары попадут в рекомендательную систему. Некоторые из них встречаются довольно редко, поэтому мы обязаны "отсечь этот хвост".\
Для этого прибегнем к статистическим методам и определим, какое количество купленных курсов считать низким. С этим поможет метод процентилей.

1. Вычисление минимальной границы показателей

In [40]:
# Создаем DataFrame с уникальными курсами
unique_courses = pd.DataFrame(set(df["resource_id"]))
unique_courses.columns = ["course"]

# Создаем DataFrame, в котором посчитаем количество купленных пар курсов
freq_df = pd.DataFrame(unique_courses, index=pair_counts.keys(), columns=["freq"])
freq_df["freq"] = pair_counts.values()

# При помощи статистического метода percentile отфильтруем менее популярные пары (выбираем 85-ый процентиль за порог минимальной частоты)
min_freq = np.percentile(freq_df["freq"], 85)
min_freq

np.float64(15.0)

Минимальная граница определена - 15. Приступим к созданию таблицы, в которую будем добавлять курсы через итерацию, учитывая минимальный показатель.

2. Создание функции, которая для каждого введенного id-курса будет возвращать 2 курса, с которыми поданный на вход курс чаще всего продавался

In [41]:
# Создаем список для дальнейшего построения таблицы рекомендаций + убираем дубликаты
list_courses = df.resource_id.drop_duplicates().tolist()
list_courses

# Создаем функцию recommendation
def recommendation(course_id):
    rec_list = []
    for i in pair_counts.keys():
        if i[0] == course_id:
            rec_list.append((i, pair_counts[i]))
        elif i[1] == course_id:
            rec_list.append((i, pair_counts[i]))
    rec_list.sort(key=lambda x: x[1], reverse=True)
    return rec_list[:2]


# К новому списку со всеми уникальными курсами применяем созданную функцию
rec_list_all = []
for i in list_courses:
    rec_list_all.append(recommendation(i))

Теперь у нас есть список со всеми уникальными курсами и парой рекомендованных к ним курсов.\
На финальном этапе воспользуемся итерацией по списку и создадим DataFrame со всеми рекомендациями

3. Создание цикла по всем уникальным курсам и запись результата в финальный DataFrame

In [42]:
# Создаем список и финальный DataFrame, куда будем помещать результаты цикла
recommendation_list = []
recomm_df = pd.DataFrame(recommendation_list, columns=["first_rec", "second_rec"])

# Создание цикла, который пройдется по частотности пары 1-ой и 2-ой рекомендации, учитывая min_freq (минимально допустимый показатель частотности)
for course_id in list_courses:
    rec1 = None
    rec2 = None
    rec = recommendation(course_id)
    if rec[0][1] >= min_freq:
        rec1 = (set(rec[0][0]) - set([course_id])).pop()
    if rec[1][1] >= min_freq:
        rec2 = (set(rec[1][0]) - set([course_id])).pop()
    recomm_df.loc[course_id] = [rec1, rec2]
    # При отсутствии значений будем использовать "запасные" значения 551 и 566
    if (
        recomm_df.loc[course_id]["first_rec"] is None
        and recomm_df.loc[course_id]["second_rec"] is None
    ):
        recomm_df.loc[course_id] = [551, 566]
    if rec2 is None and rec1 is not None:
        rec1 = (set(rec[0][0]) - set([course_id])).pop()
        rec2 = np.random.choice(list(set([551, 566]) - set([course_id])))
        recomm_df.loc[course_id] = [rec1, rec2]

recomm_df

,first_rec,second_rec
516,745,553
1099,745,568
356,571,357
357,571,356
1125,571,912
...,...,...
902,551,566
837,551,566
1200,551,566
833,551,566


### ИТОГИ:

Таким образом, мы получили рекомендательную систему, состоящую из 3 колонок:
1) Курс, к которому подбирается рекомендация;
2) Первый курс (самый популярный) для рекомендации;
3) Второй курс (по популярности) для рекомендации.

Следует прояснить один момент с количеством получившихся пар - 126. В начале выгрузки датасета на вопрос "Сколько всего уникальных курсов?" мы получили ответ - 127. Однако, как показывает таблица, всего насчитывается 126 уникальных курсов, что можно объяснить так: один из курсов еще ни разу не был куплен, поэтому и не попал в нашу выборку.

Итоговая таблица передается для дальнейшей работы продакт-менеджеру онлайн-школы MasterMind.

Еще одним незакрытым вопросом является обязательное проведение А/В-тестирования после внедрения новой разработки с целью определения ее эффективности.

